# TNT Loop-Bridge: Creep Compliance
> **Handbook:** This notebook demonstrates the TNT Loop-Bridge model. For complete mathematical derivations and theoretical background, see [TNT Loop-Bridge Documentation](../../docs/source/models/tnt/tnt_loop_bridge.rst).


**Estimated Time:** 4-6 minutes

## Protocol: Creep in Loop-Bridge Networks

**Creep** in loop-bridge systems shows **complex compliance** with delayed loop-bridge conversion under sustained stress.

### Physical Context

Under constant stress:
1. **Immediate elastic**: Both loops and bridges stretch ($J_e = 1/G_{total}$)
2. **Transient conversion**: Stress drives loop→bridge or bridge→loop redistribution
3. **Steady flow**: New equilibrium bridge fraction determines final viscosity

The creep rate depends on applied stress magnitude through force-dependent conversion kinetics.

> **Loop-Bridge Creep Physics**  
> For stress-dependent conversion and compliance analysis, see:  
> [../../docs/source/models/tnt/tnt_loop_bridge.rst](../../docs/source/models/tnt/tnt_loop_bridge.rst)

---

## Learning Objectives

1. Understand stress-dependent loop-bridge equilibrium
2. Fit TNT Loop-Bridge model to creep compliance data
3. Extract conversion rates from transient regime
4. Analyze stress-dependent viscosity from steady creep
5. Compare loop-bridge to Cates creep (different nonlinearity)
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- TNT Loop-Bridge fundamentals (Notebook 13)
- Understanding of creep (TNT Single-Mode NB04)

## Runtime Estimate

- NLSQ fitting: ~8-15 seconds
- Bayesian inference: ~90-180 seconds
- Total: ~8-12 minutes

## Setup

In [ ]:
import os
import sys
import time

# CI mode for automated testing
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import cumulative_trapezoid

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64

verify_float64()

from rheojax.models.tnt import TNTLoopBridge

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    compute_bell_effective_lifetime,
    compute_fit_quality,
    compute_maxwell_moduli,
    get_tnt_loop_bridge_param_names,
    load_epstein_saos,
    load_laponite_relaxation,
    load_ml_ikh_creep,
    load_ml_ikh_flow_curve,
    load_pnas_laos,
    load_pnas_startup,
    plot_bell_nu_sweep,
    plot_loop_bridge_fraction,
    print_convergence_summary,
    print_nu_interpretation,
    print_parameter_comparison,
    save_tnt_results,
)

param_names = get_tnt_loop_bridge_param_names()

from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Creep Dynamics

### Physical Picture

Under constant stress:
1. **Elastic response**: Immediate strain gamma_0 = sigma / G_eff
2. **Transient creep**: Bridge fraction evolves, effective modulus changes
3. **Steady flow**: If stress exceeds yield threshold, continuous strain accumulation

### Governing Equations

**Stress Balance (Maxwell backbone):**
```
sigma = f_B * G * gamma_e
d(gamma_e)/dt = gamma_dot - gamma_e / tau_b
```

**Bridge Fraction Evolution:**
```
df_B/dt = (1 - f_B)/tau_a - f_B * exp(nu * gamma_dot * tau_b) / tau_b
```

**Total Strain:**
```
gamma(t) = gamma_e(t) + int_0^t gamma_dot(s) ds
```

**Compliance:**
```
J(t) = gamma(t) / sigma
```

### Stress-Dependent Bridge Fraction

At constant stress, the shear rate adjusts to maintain stress balance:
```
gamma_dot(t) = sigma / (f_B(t) * G * tau_b)
```

Higher stress → higher gamma_dot → stronger Bell detachment → lower f_B → higher gamma_dot (positive feedback)

### Delayed Yielding

If stress is close to the dynamic yield stress, the material may exhibit:
- Initial slow creep (f_B ~ f_B_eq)
- Accelerating creep (f_B decreases)
- Eventual flow (f_B stabilizes at low value)

### Compliance Evolution

Compliance J(t) increases due to:
1. Elastic compliance: J_e = 1 / (f_B * G)
2. Viscous flow: dJ/dt = gamma_dot / sigma

As f_B decreases, J_e increases, accelerating compliance growth.

## Load Creep Data

In [ ]:
time_data, shear_rate, _, stress_applied = load_ml_ikh_creep(stress_pair_index=0)

# Integrate shear rate to get strain
strain = cumulative_trapezoid(shear_rate, time_data, initial=0)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} - {time_data.max():.2e} s")
print(f"Applied stress: {stress_applied:.2f} Pa")
print(f"Strain range: {strain.min():.4f} - {strain.max():.4f}")
print(f"Final shear rate: {shear_rate[-1]:.4e} 1/s")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Strain vs time
ax1.plot(time_data, strain, 'o', label='Data', markersize=6)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Strain', fontsize=12)
ax1.set_title(f'Creep Data (σ = {stress_applied:.2f} Pa)', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Compliance vs time
compliance = strain / stress_applied
ax2.loglog(time_data, compliance, 'o', label='Data', markersize=6)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Compliance J(t) (1/Pa)', fontsize=12)
ax2.set_title('Creep Compliance', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTLoopBridge()

if FAST_MODE:
    print("FAST_MODE: Using default parameters (NLSQ fit for creep is slow)")
    # Set reasonable default parameters for TNTLoopBridge creep
    model.parameters.set_value('G', 100.0)
    model.parameters.set_value('tau_b', 1.0)
    model.parameters.set_value('tau_a', 0.1)
    model.parameters.set_value('nu', 0.5)
    model.parameters.set_value('f_B_eq', 0.5)
    model.parameters.set_value('eta_s', 0.01)
    t_nlsq = 0.0
else:
    print("Starting NLSQ fit...")
    t_start = time.time()

    nlsq_result = model.fit(time_data, strain, test_mode='creep', sigma_applied=stress_applied, method='scipy')

    t_nlsq = time.time() - t_start

print(f"\nNLSQ fit completed in {t_nlsq:.2f} seconds")
print(f"\nFitted parameters:")
for name in param_names:
    value = model.parameters.get_value(name)
    print(f"  {name}: {value:.4e}")

strain_pred_fit = model.predict(time_data, test_mode='creep', sigma_applied=stress_applied)
metrics = compute_fit_quality(strain, strain_pred_fit)
print(f"\nFit quality:")
print(f"  R²: {metrics['R2']:.6f}")
print(f"  RMSE: {metrics['RMSE']:.4e}")

## NLSQ Fit Visualization

In [ ]:
# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    time_data, strain, model, test_mode="creep",
    param_names=param_names, log_scale=True,
    xlabel='Time (s)',
    ylabel=r'Strain $\gamma$',
    title=f'NLSQ Fit (R\u00b2 = {metrics["R2"]:.4f})',
    sigma_applied=stress_applied
)
plt.close("all")

## Physical Analysis: Stress-Dependent Bridge Fraction

In [ ]:
# Create prediction grid
time_pred = np.linspace(time_data.min(), time_data.max(), 200)
strain_pred = model.predict(time_pred, test_mode='creep', sigma_applied=stress_applied)

# Estimate effective shear rate from strain growth
strain_rate_pred = jnp.gradient(strain_pred, time_pred)

# Estimate bridge fraction from stress balance
# sigma = f_B * G * gamma_dot * tau_b
# f_B = sigma / (G * gamma_dot * tau_b)
f_B_est = stress_applied / (model.parameters.get_value('G') * strain_rate_pred * model.parameters.get_value('tau_b') + 1e-10)  # avoid division by zero
f_B_est = jnp.clip(f_B_est, 0.0, 1.0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Strain rate evolution
ax1.semilogy(time_pred, strain_rate_pred, '-', linewidth=2)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Shear Rate (1/s)', fontsize=12)
ax1.set_title('Shear Rate Evolution During Creep', fontsize=14)
ax1.grid(True, alpha=0.3)

# Bridge fraction estimate
ax2.plot(time_pred, f_B_est, '-', linewidth=2)
ax2.axhline(model.parameters.get_value('f_B_eq'), color='r', linestyle='--', alpha=0.5, label=f'f_B_eq = {model.parameters.get_value('f_B_eq'):.4f}')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Bridge Fraction f_B', fontsize=12)
ax2.set_title('Bridge Fraction Evolution (Estimated)', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1])

plt.tight_layout()
plt.close("all")
plt.close('all')

print(f"\nCreep dynamics:")
print(f"  Initial shear rate: {strain_rate_pred[0]:.4e} 1/s")
print(f"  Final shear rate: {strain_rate_pred[-1]:.4e} 1/s")
print(f"  Acceleration factor: {strain_rate_pred[-1] / strain_rate_pred[0]:.2f}")
print(f"  Estimated initial f_B: {f_B_est[0]:.4f}")
print(f"  Estimated final f_B: {f_B_est[-1]:.4f}")

## Physical Analysis: Compliance Evolution

In [ ]:
# Compliance from model prediction
compliance_pred = strain_pred / stress_applied

# Elastic compliance estimate
J_e = 1.0 / (model.parameters.get_value('G') * model.parameters.get_value('f_B_eq'))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Compliance vs time
ax1.loglog(time_data, compliance, 'o', label='Data', markersize=6, alpha=0.7)
ax1.loglog(time_pred, compliance_pred, '-', label='NLSQ Fit', linewidth=2)
ax1.axhline(J_e, color='r', linestyle='--', alpha=0.5, label=f'J_e = {J_e:.4e} 1/Pa')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Compliance J(t) (1/Pa)', fontsize=12)
ax1.set_title('Creep Compliance', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Compliance growth rate
dJ_dt = jnp.gradient(compliance_pred, time_pred)
ax2.loglog(time_pred[1:], dJ_dt[1:], '-', linewidth=2)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('dJ/dt (1/(Pa·s))', fontsize=12)
ax2.set_title('Compliance Growth Rate', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

print(f"\nCompliance analysis:")
print(f"  Elastic compliance J_e: {J_e:.4e} 1/Pa")
print(f"  Initial compliance: {compliance_pred[0]:.4e} 1/Pa")
print(f"  Final compliance: {compliance_pred[-1]:.4e} 1/Pa")
print(f"  Compliance growth: {compliance_pred[-1] / compliance_pred[0]:.2f}x")

## Physical Analysis: Stress Sweep

In [ ]:
if FAST_MODE:
    print("FAST_MODE: Skipping stress sweep (multiple ODE predictions are slow)")
else:
    # Simulate creep at different stress levels
    stress_sweep = jnp.array([stress_applied * 0.5, stress_applied, stress_applied * 1.5, stress_applied * 2.0])
    time_sweep = jnp.linspace(0, time_data.max(), 200)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    for sigma in stress_sweep:
        strain_sweep = model.predict(time_sweep, test_mode='creep', stress=sigma)
        ax1.plot(time_sweep, strain_sweep, '-', linewidth=2, label=f'σ = {sigma:.2f} Pa')

    ax1.set_xlabel('Time (s)', fontsize=12)
    ax1.set_ylabel('Strain', fontsize=12)
    ax1.set_title('Creep Curves at Different Stresses', fontsize=14)
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)

    # Final strain vs stress
    final_strains = []
    for sigma in stress_sweep:
        strain_sweep = model.predict(time_sweep, test_mode='creep', stress=sigma)
        final_strains.append(strain_sweep[-1])

    ax2.plot(stress_sweep, final_strains, 'o-', linewidth=2, markersize=8)
    ax2.set_xlabel('Stress (Pa)', fontsize=12)
    ax2.set_ylabel('Final Strain', fontsize=12)
    ax2.set_title('Final Strain vs Applied Stress', fontsize=14)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.close("all")
    plt.close('all')

    print(f"\nStress dependence:")
    for i, sigma in enumerate(stress_sweep):
        print(f"  σ = {sigma:.2f} Pa: final strain = {final_strains[i]:.4f}")

## Bayesian Inference

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayes_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayes_result = model.fit_bayesian(
        time_data, strain,
        test_mode='creep',
        sigma_applied=stress_applied,
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(bayes_result, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## Parameter Comparison: NLSQ vs Bayesian

In [ ]:
print_parameter_comparison(model, bayes_result.posterior_samples, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayes_result, 'to_inference_data'):
    display_arviz_diagnostics(bayes_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayes_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        time_data,
        strain,
        model, bayes_result, test_mode="creep",
        param_names=param_names, log_scale=True,
        xlabel=r'Time (s)',
        ylabel=r'Strain $\\gamma$', sigma_applied=stress_applied
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation

In [ ]:
print("\n=== Physical Interpretation ===")
print(f"\n1. Material Properties:")
print(f"   - Plateau modulus G: {model.parameters.get_value('G'):.4e} Pa")
print(f"   - Equilibrium bridge fraction: {model.parameters.get_value('f_B_eq'):.4f}")
print(f"   - Elastic compliance: {J_e:.4e} 1/Pa")

print(f"\n2. Creep Conditions:")
print(f"   - Applied stress: {stress_applied:.4e} Pa")
print(f"   - Test duration: {time_data.max():.4e} s")
print(f"   - Final strain: {strain[-1]:.4f}")
print(f"   - Final compliance: {compliance[-1]:.4e} 1/Pa")

print(f"\n3. Kinetic Timescales:")
print(f"   - Bridge detachment time tau_b: {model.parameters.get_value('tau_b'):.4e} s")
print(f"   - Loop attachment time tau_a: {model.parameters.get_value('tau_a'):.4e} s")
print(f"   - Ratio tau_a/tau_b: {model.parameters.get_value('tau_a')/model.parameters.get_value('tau_b'):.4f}")

print(f"\n4. Flow Behavior:")
print(f"   - Initial shear rate: {strain_rate_pred[0]:.4e} 1/s")
print(f"   - Final shear rate: {strain_rate_pred[-1]:.4e} 1/s")
print(f"   - Acceleration: {strain_rate_pred[-1] / strain_rate_pred[0]:.2f}x")
if strain_rate_pred[-1] > strain_rate_pred[0] * 2:
    print(f"   - Accelerating creep detected (possible delayed yielding)")
else:
    print(f"   - Steady creep behavior")

print(f"\n5. Bridge Depletion:")
print(f"   - Estimated initial f_B: {f_B_est[0]:.4f}")
print(f"   - Estimated final f_B: {f_B_est[-1]:.4f}")
print(f"   - Bridge fraction loss: {(1 - f_B_est[-1]/f_B_est[0])*100:.2f}%")

print(f"\n6. Bell Detachment:")
print(f"   - Nu parameter: {model.parameters.get_value('nu'):.4f}")
gamma_dot_avg = strain[-1] / time_data[-1]
force_factor = jnp.exp(model.parameters.get_value('nu') * gamma_dot_avg * model.parameters.get_value('tau_b'))
print(f"   - Average shear rate: {gamma_dot_avg:.4e} 1/s")
print(f"   - Bell force factor: {force_factor:.4f}")
print(f"   - Detachment enhancement: {force_factor:.2f}x")

## Save Results

In [ ]:
save_tnt_results(model, bayes_result, "loop_bridge", "creep", param_names)
print("Results saved to reference_outputs/tnt/loop_bridge_creep_results.npz")

## Key Takeaways

1. **Stress-Dependent Kinetics**: Higher stress → higher shear rate → stronger Bell detachment → lower f_B

2. **Delayed Yielding**: Material may exhibit slow initial creep followed by acceleration as f_B decreases

3. **Compliance Growth**: J(t) increases due to both elastic (1/G_eff) and viscous (integral of gamma_dot) contributions

4. **Positive Feedback**: Bridge depletion → reduced modulus → higher shear rate → more depletion

5. **Timescale Control**: tau_b and tau_a govern creep rate and recovery potential

6. **Bell Physics**: Force-enhanced detachment (nu > 0) is crucial for stress-dependent creep

7. **Experimental Challenge**: Creep data requires careful integration, numerical noise amplifies in shear rate

## Next Steps

- **Notebook 17**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
